# Real program

Structure of url :
https://www.leboncoin.fr/annonces/offres/rhone_alpes/?th=1&q=batterie%20lithium&it=0&parrot=1
region : /rhone_alpes/
keywords : q=batterie%20lithium
look only in title : it=1
only urgent ones : ur=1
category : https://www.leboncoin.fr/locations/offres/rhone_alpes/?th=1&q=batterie%20lithium&parrot=1
city : location=Toutes%20les%20communes%2073000


In [1]:
from urllib.request import urlopen
from lxml import etree

# ====== we define the parameters of the request

l_keywords = ['batterie', 'lithium']
keywords = '%20'.join(l_keywords)

region = "rhone_alpes"
code_postal = "73000"
#location = "&location=Toutes%20les%20communes%20" + code_postal
location = ""


In [2]:
import re # regular expressions

class ad_class:
    def __init__(self, title, price, url, time, city, img):
        self.title = title
        self.price = price
        self.url = url
        self.time = time
        self.city = city
        self.img = img
        
        self.ah = self.return_ah(self.title)
    # first evaluation to see if the ad is interesting
    # returns True if the ad is interesting
    def looks_interesting(self):
        if float(self.price) > 300:
            return False
        print(str(self.ah / self.price) + " ah/euros")
        return True
    def return_ah(self, string):
        pattern = re.compile(r'([0-9.,]*\Smah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-3].replace(",", ".")
            return float(ah)/1000
        pattern = re.compile(r'([0-9.,]* mah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-4].replace(",", ".")
            return float(ah)/1000
        pattern = re.compile(r'([0-9.,]*\Sah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-2].replace(",", ".")
            return float(ah)
        pattern = re.compile(r'([0-9.,]* ah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-3].replace(",", ".")
            return float(ah)

class page_class:
    def __init__(self, url_request):
        htmlparser = etree.HTMLParser()
        page_ads = urlopen(url_request)
        self.tree_ads = etree.parse(page_ads, htmlparser)
    def get_infos_ad(self, i):
        ad_glob = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a")
        ad_url = "http:" + ad_glob[0].values()[0]
        ad_title = ad_glob[0].values()[1]

        ad_img = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a/div[1]/span[1]/span[1]")
        url_img_ad = "http:" + ad_img[0].values()[2]

        ad_city_elmt = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a/section/p[2]")
        ad_city = ad_city_elmt[0].getchildren()[0].values()

        ad_price_elmt = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a/section/h3")
        ad_price = ad_price_elmt[0].values()[2]

        ad_time_elmt = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[1]/a/section/aside/p")
        ad_time = ad_time_elmt[0].values()[2]
        
        return ad_class(ad_title, ad_price, ad_url, ad_time, ad_city, url_img_ad)
        

In [3]:

# === loop on the pages
o = 1

url_request = "https://www.leboncoin.fr/annonces/offres/" + region + "/?o=" + str(o) + "&th=1&q=" + keywords + "&it=1" + location + "&parrot=1"
#page_ads = urlopen(url_request)
#tree_ads = etree.parse(page_ads, htmlparser)
page = page_class(url_request)


In [5]:
# == loop on the ads
ok = True
i = 0
while(ok):
    i = i + 1
    try:
        ad_infos = page.get_infos_ad(i)
        if (ad_infos.ah != None):
            print(ad_infos.ah / float(ad_infos.price), ad_infos.url)
    except:
        ok = False

0.1 http://www.leboncoin.fr/bricolage/1123463756.htm?ca=22_s
0.05 http://www.leboncoin.fr/outillage_materiaux_2nd_oeuvre/1121432571.htm?ca=22_s
0.4 http://www.leboncoin.fr/bricolage/1114115173.htm?ca=22_s
0.31 http://www.leboncoin.fr/telephonie/793271761.htm?ca=22_s
0.05 http://www.leboncoin.fr/bricolage/1119986762.htm?ca=22_s
0.044444444444444446 http://www.leboncoin.fr/velos/1119858366.htm?ca=22_s
0.05357142857142857 http://www.leboncoin.fr/restauration_hotellerie/1089578522.htm?ca=22_s
